# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('weather_data.csv')
weather_df

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.15,-70.92,33.80,59,61,8.05,CL,1595904358
1,1,hithadhoo,-0.60,73.08,82.87,78,100,14.05,MV,1595904370
2,2,kungurtug,50.60,97.52,69.33,38,99,2.10,RU,1595904678
3,3,fort myers beach,26.45,-81.95,81.00,74,1,10.29,US,1595904582
4,4,tuktoyaktuk,69.45,-133.04,59.00,77,90,13.87,CA,1595904375
...,...,...,...,...,...,...,...,...,...,...
575,575,mikuni,36.22,136.15,84.20,70,90,13.87,JP,1595904722
576,576,fort-shevchenko,44.51,50.26,79.61,37,0,17.47,KZ,1595904722
577,577,porto novo,6.50,2.60,75.20,94,20,11.41,BJ,1595904722
578,578,nkhotakota,-12.93,34.30,65.64,76,86,11.88,MW,1595904722


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_df[['Lat','Lon']]
humidity = weather_df['Humidity']

fig = gmaps.figure()
heat = gmaps.heatmap_layer(locations,weights=humidity,
                           dissipating=False,  
                           max_intensity=100, point_radius=5)
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
trim =  (weather_df['Wind Speed']>=10) & (weather_df['Cloudiness']==0) & ((weather_df['Max Temp']>=69) | (weather_df['Max Temp']<=85))
trim_df = weather_df[trim].reset_index().head(10)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = trim_df[['City','Lat','Lon','Max Temp','Humidity','Cloudiness','Wind Speed','Country','Date']]
hotel_df['Hotel Name'] = " "



C:\Users\asiha\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
for index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Lat']}, {row['Lon']}"
    target_search = "hotel"
    target_radius = 5000
    query = "the best hotel"
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "query": query,
    "key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        # convert response to json
        hotel_data = response.json()
        hotel_df.loc[index,'Hotel Name'] = hotel_data["results"][0]["name"]
    except:
        print(f"{row['City']} can not be found")




C:\Users\asiha\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


tilichiki can not be found


In [7]:
blanks = hotel_df['Hotel Name']!= " "
hotel_df = hotel_df[blanks]
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,atuona,-9.80,-139.03,79.74,71,0,25.70,PF,1595904204,Simplicité Marquises
1,pevek,69.70,170.31,35.31,86,0,11.45,RU,1595904203,Munitsipal'naya Gostinitsa
2,lebu,-37.62,-73.65,44.06,77,0,11.50,CL,1595904351,Hostal Plaza Lebu
3,carnarvon,-24.87,113.63,71.60,56,0,14.99,AU,1595904353,Carnarvon Motel WA
4,yulara,-25.24,130.99,62.60,33,0,14.99,AU,1595904401,Sails in the Desert
5,trelew,-43.25,-65.31,40.60,68,0,17.16,AR,1595904688,Rayentray Trelew
7,safaga,26.73,33.94,83.80,49,0,16.06,EG,1595904689,Lamar Resort Abu Soma
8,vestmannaeyjar,63.44,-20.27,50.13,77,0,19.71,IS,1595904196,Hotel Vestmannaeyjar
9,guerrero negro,27.98,-114.06,67.86,85,0,18.70,MX,1595904365,Hotel Los Caracoles


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure


fig

Figure(layout=FigureLayout(height='420px'))